In [ ]:
# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib import colors as mcolors
from math import 
# from matplotlib import colors as mcolors
from pathlib import Path

import package_DBR
from package_DBR import myRound, SelectPath_RT, Delay_RT, FO_RT, FOPDT, SOPDT, FOPDT_cost, SOPDT_cost, Process, Bode

In [ ]:
# Load Data

In [ ]:
nameFile = 'Cleaned_data_Open_loop_experiment_on_MV_2025-03-10-13h46.txt'

titleName = nameFile.split('.')[0]  
data = pd.read_csv('Data/' + nameFile)



if "MV" in nameFile :
    ExpVariable = "MV"
    tm = data['tm'].values# - data['t'].values[0]
    MVm = data['MVm'].values
    PVm = data['PVm'].values

else:
    ExpVariable = "DV"
    tm = data['tm'].values
    DVm = data['DVm'].values
    PVm = data['PVm'].values

print(ExpVariable)


In [ ]:
# Approximations Methods

In [ ]:
### Parameters (obtained via Tangent at Inflexion Method)

In [ ]:
# Méthode de la Tangente au point d'inflexion

if "MV" in nameFile :
    Kp = 0.37             # Process gain (affect MV and PV)
    T1 = 95   # same == // 
    T2 = 126  # experimental value maybe, i dont know if this one or other
    a = 0.065
    T_g = 186
    T_u = 26 
    
else:
    Kd = 0            #  Disturbance gain : Affects the impact of the disturbance on PV.
    T1_d = 0            #  how quickly the disturbance affects the PV.
    T2_d = 0 
    a = 0.1
    T_g = 0
    T_u = 0

print('VDG_Ratio = ', T_u/T_g)


Ts = 1  # Value Imposed
TSim = len(MVm)




In [ ]:
# Plot loaded Data

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1)
fig.set_figheight(12)
fig.set_figwidth(22)

l1, = ax1.step([0,1],[0,100],'b-',linewidth=2,label='MV',where='post')
ax1.set_ylabel('Value of MV [%]')
ax1.set_title(titleName)
ax1.legend(loc='best')

l2, = ax2.step([0,1],[0,100],'g-',linewidth=2,label='PV',where='post')
ax2.set_ylabel('Value of PV [°C]')
ax2.legend(loc='best')

l3, = ax3.step([0,1],[0,100],'r-',linewidth=2,label='DV',where='post')
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Value of DV [%]')
ax3.legend(loc='best')

l1.set_data(t,MV)
l2.set_data(t,PV)
l3.set_data(t,DV)

ax1.set_xlim(0, t[-1]+1)
ax2.set_xlim(0, t[-1]+1)
ax3.set_xlim(0, t[-1]+1)

ax1.set_ylim(myRound(np.min(MV),5)-5, myRound(np.max(MV),5)+5)
ax2.set_ylim(myRound(np.min(PV),5)-5, myRound(np.max(PV),5)+5)
ax3.set_ylim(myRound(np.min(DV),5)-5, myRound(np.max(DV),5)+5)

In [ ]:
# FOPDT (Simple Broida Method)

In [ ]:
#Broida1
T = T_g
theta = T_u
num = K_P*e**((-theta)*s)
den = T*s + 1

P = num/den

#FOPDT(MV,Kp,T,theta,Ts,MVInit=0,PVInit=0,method='EBD'):
PV_Broida_EASY = FOPDT(MV=MVm, Kp=Kp_Broida_EASY, T=T_Broida_EASY, theta=theta_Broida_EASY, Ts=Ts, PVInit=0, method='EBD')


#============PLOT===========================
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,ncols=1, sharex='col', gridspec_kw={'height_ratios': [1.5, 7, 7]})
fig.set_figheight(12)
fig.set_figwidth(20)

ax1.set_title("Broida's Simple Approximation")

#--MV
limSup = 1.5
limInf = -0.5
l1, = ax1.step([0,TSim], [limInf,limSup], 'green',linewidth=2, label='MVm', where="post")
ax1.set_ylabel('MV')
ax1.legend (loc='best')

#----PV
limSup = 0.45
limInf = 0
l2, = ax2.step([0, TSim], [limInf,limSup], 'blue', linewidth=3, label='PVm', where='post') 
l3, = ax2.step([0, TSim], [limInf,limSup], 'green', linewidth=3, label='PV_Broida_EASY', where='post') 
ax2.set_ylabel('PV]')
ax2.legend (loc='best')

#---Model_Diff
limSup = 0.1
limInf = -0.1
diff_BroidaEASY_PV = [a - b for a,b in zip(PVm, PV_Broida_EASY)]
l4, = ax3.step([0, TSim], [limInf,limSup], 'red', linewidth=3, label='Model_Diff', where='post') 
ax3.set_ylabel('delta')
ax3.legend (loc='best')


#-----Set Data-----
l1.set_data(tm, MVm)

l2.set_data(tm,PVm)
l3.set_data(tm, PV_Broida_EASY)

l4.set_data(tm,diff_BroidaEASY_PV)


tMax = len(tm)
ax1.set_xlim(0, tMax)
ax2.set_xlim(0, tMax)
ax3.set_xlim(0, tMax)

ax1.grid()
ax2.grid()
ax3.grid()






In [ ]:
# FOPDT (Broida's Harder Method)

In [ ]:
#Broida2
T = 5.5*(t_2 - t_1)
theta = 2.8*t_1 - 1.8*t_2
num = K_P*e**((-theta)*s)
den = T*s + 1

P = num/den

#FOPDT(MV,Kp,T,theta,Ts,MVInit=0,PVInit=0,method='EBD'):
PV_Broida_HARD = FOPDT(MV=MVm, Kp=Kp_Broida_HARD, T=T_Broida_HARD, theta=theta_Broida_HARD, Ts=Ts, PVInit=0, method='EBD')


#============PLOT===========================
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,ncols=1, sharex='col', gridspec_kw={'height_ratios': [1.5, 7, 7]})
fig.set_figheight(12)
fig.set_figwidth(20)

ax1.set_title("Broida's HARD Approximation")

#--MV
limSup = 1.5
limInf = -0.5
l1, = ax1.step([0,TSim], [limInf,limSup], 'green',linewidth=2, label='MVm', where="post")
ax1.set_ylabel('MV')
ax1.legend (loc='best')

#----PV
limSup = 0.45
limInf = 0
l2, = ax2.step([0, TSim], [limInf,limSup], 'blue', linewidth=3, label='PVm', where='post') 
l3, = ax2.step([0, TSim], [limInf,limSup], 'green', linewidth=3, label='PV_Broida_HARD', where='post') 
ax2.set_ylabel('PV')
ax2.legend (loc='best')

#---Model_Diff
limSup = 0.05
limInf = -0.05
diff_BroidaEASY_PV = [a - b for a,b in zip(PVm, PV_Broida_HARD)]
l4, = ax3.step([0, TSim], [limInf,limSup], 'red', linewidth=3, label='Model_Diff', where='post') 
ax3.set_ylabel('Delta')
ax3.legend (loc='best')


#-----Set Data-----
l1.set_data(tm, MVm)

l2.set_data(tm,PVm)
l3.set_data(tm, PV_Broida_HARD)

l4.set_data(tm,diff_BroidaEASY_PV)


tMax = len(tm)
ax1.set_xlim(0, tMax)
ax2.set_xlim(0, tMax)
ax3.set_xlim(0, tMax)

ax1.grid()
ax2.grid()
ax3.grid()







In [ ]:
SOPDT (Van Der Grinten)

In [22]:
#Van der Grinten
T_1 = T_g*(3*a*e - 1)/(1+a*e)
T_2 = T_g*(1-a*e)/(1+a*e)
theta = T_u - (T_1*T_2)/(T_1 + 3*T_2)





In [ ]:
#Strejc
ratio = T_u/T_g

if ratio >= 0 and ratio < 0.1 :
    n = 1
elif ratio >= 0.1 and ratio < 0.22 :
    n = 2
elif ratio >= 0.22 and ratio < 0.32 :
    n = 3
elif ratio >= 0.32 and ratio < 0.41 :
    n = 4
elif ratio >= 0.41 and ratio < 0.49 :
    n = 5
elif ratio >= 0.49 and ratio < 0.57 :
    n = 6
elif ratio >= 0.57 :
    n = 7

if n == 1 :
    a_n = 0
elif n == 2 :
    a_n = 0.1
elif n == 3 :
    a_n = 0.22
elif n == 4 :
    a_n = 0.32
elif n == 5 :
    a_n = 0.1